In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [10]:
data_treino = pd.read_csv('/home/caio/github/k-3/data/train.csv')
data_teste = pd.read_csv('/home/caio/github/k-3/data/test.csv')

## Pré-Processamento

### Normalizando Dewpoint

In [3]:
data_treino['dewpoint'] = data_treino['dewpoint'].interpolate()
data_teste['dewpoint'] = data_treino['dewpoint'].interpolate()

### Normalizando Winddirection

In [4]:
data_teste['winddirection'] = data_teste['winddirection'].fillna(104.863151)

### Diminuindo Colunas Maxtemp, Temparature e mintemp (PCA)

In [5]:
# Selecionando as Colunas
colunas = ['maxtemp', 'temparature', 'mintemp', 'dewpoint']
reducao = data_treino[colunas]

# Normalizando os dados (PCA funciona melhor com dados padronizados)
scaler = StandardScaler()
reducao_scalada = scaler.fit_transform(reducao)

# Aplicar PCA
pca = PCA(n_components=1)  # Reduzindo para 2 componentes
reducao_unificada = pca.fit_transform(reducao_scalada)

# Criar um DataFrame com os novos componentes
temp1= pd.DataFrame(reducao_unificada, columns=['PCA1_A'])

print(pca.explained_variance_ratio_)

[0.96481135]


In [8]:
# Selecionando as Colunas
reducao = data_teste[colunas]

# Normalizando os dados (PCA funciona melhor com dados padronizados)
scaler = StandardScaler()
reducao_scalada = scaler.fit_transform(reducao)

# Aplicar PCA
pca = PCA(n_components=1)  # Reduzindo para 2 componentes
reducao_unificada = pca.fit_transform(reducao_scalada)

# Criar um DataFrame com os novos componentes
temp2 = pd.DataFrame(reducao_unificada, columns=['PCA1_B'])

print(pca.explained_variance_ratio_)

KeyError: "None of [Index(['maxtemp', 'temparature', 'mintemp', 'dewpoint'], dtype='object')] are in the [columns]"

In [7]:
data_treino = data_treino.drop(columns=colunas)  # Remover variáveis originais
data_teste = data_teste.drop(columns=colunas)
data_treino['Temp_unificada'] = temp1['PCA1_A']  # Adicionar a nova variável PCA1
data_teste['Temp_unificada'] = temp2['PCA1_B']

data_treino.head()

,id,day,pressure,humidity,cloud,sunshine,winddirection,windspeed,rainfall,Temp_unificada
0,0,1,1017.4,87.0,88.0,1.1,60.0,17.2,1,-1.105581
1,1,2,1019.5,95.0,91.0,0.0,50.0,21.9,1,-2.685513
2,2,3,1024.1,75.0,47.0,8.3,70.0,18.1,1,-3.164962
3,3,4,1013.4,95.0,95.0,0.0,60.0,35.6,1,-2.190972
4,4,5,1021.8,52.0,45.0,3.6,40.0,24.8,0,-2.686549


## CatBoost

## Função para salvar novos Dataframes

In [9]:
destino_treino = '/home/caio/github/k-3/data/train_tratado.csv'
destino_teste = '/home/caio/github/k-3/data/teste_tratado.csv'

data_treino.to_csv(destino_treino, index=False)
data_teste.to_csv(destino_teste, index=False)